In [1]:
from __future__ import print_function
from __future__ import division
import argparse
import os
import copy
import time

import numpy as np

import torch
import torch.utils.data
from torch import nn
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import torchvision.models as models

import torch.optim as optim
from torch.optim import lr_scheduler

from SemiSupervised import SemiSupervised
from modelVae_linear import Linear_Model
from modelVae_convnet import Conv_Model
from base_model import Base_Model

In [2]:
PATH = "./weights/baseline_check2.pt"

In [7]:
checkpoint = torch.load(PATH, map_location='cpu')
print(checkpoint['epoch'])

9


In [9]:
torch.save(checkpoint['model_state_dict'], "./baseline_weights.pt")

In [17]:
#parser = argparse.ArgumentParser(description='VAE MNIST Example')
#parser.add_argument('--batch-size', type=int, default=128, metavar='N',
#                    help='input batch size for training (default: 128)')
#parser.add_argument('--epochs', type=int, default=10, metavar='N',
#                    help='number of epochs to train (default: 10)')
#parser.add_argument('--no-cuda', action='store_true', default=False,
#                    help='enables CUDA training')
#parser.add_argument('--seed', type=int, default=1, metavar='S',
#                    help='random seed (default: 1)')
#parser.add_argument('--log-interval', type=int, default=10, metavar='N',
#                    help='how many batches to wait before logging training status')
#args = parser.parse_args()
#args.cuda = not args.no_cuda and torch.cuda.is_available()

#torch.manual_seed(args.seed)

#device = torch.device("cuda" if args.cuda else "cpu")

#kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
kwargs = {}

args_batch_size = 4
semi = SemiSupervised(batch_size=args_batch_size)
train_loader = semi.load_train_data_mix(transform=transforms.ToTensor())
#test_loader = semi.load_val_data(transform=transforms.ToTensor())

Start loading mix
End loading mix


In [23]:
#%%writefile modelVae_linear.py
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

import numpy as np

from SemiSupervised import SemiSupervised

class Model(nn.Module):
    def __init__(self, x_dim=3*96*96, y_dim=3*96*96, encoder_dim=None, decoder_dim=None, latent_dim=20):
        super(Model, self).__init__()
        self.encoder_dim = []
        self.decoder_dim = []

        # Architecture
        # TODO
        # encoder_dim is a list of dimension
        if encoder_dim is None:
            self.encoder_dim = [x_dim] + [400, 20]
        else:
            self.encoder_dim = [x_dim] + encoder_dim
        
        if decoder_dim is None:
            self.decoder_dim = [latent_dim] + [20, 400]
        else:
            self.decoder_dim = [latent_dim] + decoder_dim
            
        #encoder
        #nets_en = [*self.encoder_dim]
        linear_layers_en = [nn.Linear(self.encoder_dim[i-1], self.encoder_dim[i]) for i in range(1, len(self.encoder_dim))]
        self.encoder_hidden = nn.ModuleList(linear_layers_en)
        
        # reparametrization for latent var
        in_features = self.encoder_dim[-1]
        out_features = latent_dim
        
        self.mu = nn.Linear(in_features, out_features)
        self.log_var = nn.Linear(in_features, out_features)
        
        #decode
        #nets_de = [self.encoder_dim[-1], *self.decoder_dim]
        linear_layers_de = [nn.Linear(self.decoder_dim[i-1], self.decoder_dim[i]) for i in range(1, len(self.decoder_dim))]
        
        self.decoder_hidden = nn.ModuleList(linear_layers_de)
        self.reconstruction = nn.Linear(self.decoder_dim[-1], y_dim)

        # Load pre-trained model
        # self.load_weights('weights.pth')

    def load_weights(self, pretrained_model_path, cuda=True):
        # Load pretrained model
        pretrained_model = torch.load(f=pretrained_model_path, map_location="cuda" if cuda else "cpu")

        # Load pre-trained weights in current model
        with torch.no_grad():
            self.load_state_dict(pretrained_model, strict=True)

        # Debug loading
        print('Parameters found in pretrained model:')
        pretrained_layers = pretrained_model.keys()
        for l in pretrained_layers:
            print('\t' + l)
        print('')

        for name, module in self.state_dict().items():
            if name in pretrained_layers:
                assert torch.equal(pretrained_model[name].cpu(), module.cpu())
                print('{} have been loaded correctly in current model.'.format(name))
            else:
                raise ValueError("state_dict() keys do not match")
    
    def reparametrize(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return mu + eps*std

    def forward(self, x):
        # TODO
        for layer in self.encoder_hidden:
            x = F.relu(layer(x))
        
        # reparametrization
        mu = self.mu(x)
        log_var = F.softplus(self.log_var(x))
        
        z = self.reparametrize(mu, log_var)
        
        for layer in self.decoder_hidden:
            z = F.relu(layer(z))
            
        z = self.reconstruction(z)
        return torch.sigmoid(z)

Overwriting modelVae_linear.py


In [15]:
%%writefile modelVae_convnet.py
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

import numpy as np

from SemiSupervised import SemiSupervised

class Conv_Model(nn.Module):
    def __init__(self, y_dim=3*96*96, decoder_dim=None, latent_dim=20):
        super(Model, self).__init__()
        self.decoder_dim = []

        # Architecture
        # TODO
        # encoder_dim is a list of dimension
        
        if decoder_dim is None:
            self.decoder_dim = [latent_dim] + [200, 4000]
        else:
            self.decoder_dim = [latent_dim] + decoder_dim
            
        #encoder
        #nets_en = [*self.encoder_dim]
        # input parameters: [[c_in, c_out, kernel_size, stride]]
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=1), # 16* 92 * 92
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)) # 16 * 46 * 46
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1), # 32 * 42 * 42
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=3, padding = 0)) # 32 * 14 * 14
        
        self.drop_out = nn.Dropout()
        
        self.fc1 = nn.Linear(14 * 14 * 32, latent_dim)
        
        self.encoder_hidden = nn.ModuleList([self.layer1, self.layer2, self.drop_out, self.fc1])
        
        # reparametrization for latent var
        in_features = 14 * 14 * 32
        out_features = latent_dim
        
        self.mu = nn.Linear(in_features, out_features)
        self.log_var = nn.Linear(in_features, out_features)
        
        #decode
        #nets_de = [self.encoder_dim[-1], *self.decoder_dim]
        linear_layers_de = [nn.Linear(self.decoder_dim[i-1], self.decoder_dim[i]) for i in range(1, len(self.decoder_dim))]
        
        self.decoder_hidden = nn.ModuleList(linear_layers_de)
        self.reconstruction = nn.Linear(self.decoder_dim[-1], y_dim)

        # Load pre-trained model
        # self.load_weights('weights.pth')

    def load_weights(self, pretrained_model_path, cuda=True):
        # Load pretrained model
        pretrained_model = torch.load(f=pretrained_model_path, map_location="cuda" if cuda else "cpu")

        # Load pre-trained weights in current model
        with torch.no_grad():
            self.load_state_dict(pretrained_model, strict=True)

        # Debug loading
        print('Parameters found in pretrained model:')
        pretrained_layers = pretrained_model.keys()
        for l in pretrained_layers:
            print('\t' + l)
        print('')

        for name, module in self.state_dict().items():
            if name in pretrained_layers:
                assert torch.equal(pretrained_model[name].cpu(), module.cpu())
                print('{} have been loaded correctly in current model.'.format(name))
            else:
                raise ValueError("state_dict() keys do not match")
    
    def reparametrize(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return mu + eps*std

    def forward(self, x):
        # TODO
        for layer in self.encoder_hidden:
            x = layer(x)
        
        # reparametrization
        mu = self.mu(x)
        log_var = F.softplus(self.log_var(x))
        
        z = self.reparametrize(mu, log_var)
        
        for layer in self.decoder_hidden:
            z = F.relu(layer(z))
            
        z = self.reconstruction(z)
        return torch.sigmoid(z), mu, log_var

Writing modelVae_convnet.py


In [ ]:
%%writefile infer_model.py
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import torchvision.models as models

import numpy as np

from SemiSupervised import SemiSupervised
from modelVae_linear import Linear_Model

class Infer_model(nn.Module):
    def __init__(self, ftm_path, model_path):
        self.ft = Linear_Model()
        self.ft.eval()
        self.layer1 = nn.Sequential(
            nn.Conv2d(6, 32, kernel_size=5, stride=1), # 16* 92 * 92
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)) # 16 * 46 * 46
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1), # 32 * 42 * 42
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=3, padding = 0)) # 32 * 14 * 14
        
        self.drop_out = nn.Dropout()
        
        self.fc1 = nn.Linear(14 * 14 * 32, 1000)
        
        self.classification = nn.ModuleList([self.layer1, self.layer2, self.drop_out, self.fc1])
        
    def forward(self, x):
        add_feature = self.ft(x)
        input_size = [*x.size()]
        dim = np.prod(input_size)
        
        # resize the input data
        # input data size is batch_size * 6 * 96 * 96
        input_size[1] = 2*input_size[1]
        new_data = torch.cat([x.view(-1, dim), add_feature.view(-1, dim)]).view(*input_size)
        x = new_data
        
        for layer in self.classification:
            x = layer(x)
        
        return x

In [1]:
%%writefile base_model.py
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

import numpy as np

from SemiSupervised import SemiSupervised

class Base_Model(nn.Module):
    def __init__(self, n_class = 1000):
        super(Base_Model, self).__init__()
        

        # Architecture
        # TODO
        # encoder_dim is a list of dimension
    
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=1), # 16* 92 * 92
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)) # 16 * 46 * 46
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1), # 32 * 42 * 42
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=3, padding = 0)) # 32 * 14 * 14
        
        self.drop_out = nn.Dropout()
        
        self.fc1 = nn.Linear(14 * 14 * 32, 3*n_class)
        self.fc2 = nn.Linear(3*n_class, n_class)
        
        self.encoder_hidden = nn.ModuleList([self.layer1, self.layer2, self.drop_out])

        # Load pre-trained model
        # self.load_weights('weights.pth')

    def load_weights(self, pretrained_model_path, cuda=True):
        # Load pretrained model
        pretrained_model = torch.load(f=pretrained_model_path, map_location="cuda" if cuda else "cpu")

        # Load pre-trained weights in current model
        with torch.no_grad():
            self.load_state_dict(pretrained_model, strict=True)

        # Debug loading
        print('Parameters found in pretrained model:')
        pretrained_layers = pretrained_model.keys()
        for l in pretrained_layers:
            print('\t' + l)
        print('')

        for name, module in self.state_dict().items():
            if name in pretrained_layers:
                assert torch.equal(pretrained_model[name].cpu(), module.cpu())
                print('{} have been loaded correctly in current model.'.format(name))
            else:
                raise ValueError("state_dict() keys do not match")

    def forward(self, x):
        # TODO
        for layer in self.encoder_hidden:
            x = layer(x)
            
        x = x.reshape(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        # reparametrization
        
        return x

Writing base_model.py


In [1]:
#%%writefile train.py
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import torchvision.models as models

import numpy as np
import time

from SemiSupervised import SemiSupervised
from modelVae_linear import Linear_Model

parser = argparse.ArgumentParser(description='VAE MNIST Example')
parser.add_argument('--batch-size', type=int, default=128, metavar='N',
                    help='input batch size for training (default: 128)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='enables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
args = parser.parse_args()
args.cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if args.cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}

semi = SemiSupervised(batch_size=args.batch_size)
train_loader = semi.load_train_data_mix(transform=transforms.ToTensor())
test_loader = semi.load_val_data(transform=transforms.ToTensor())

def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 3*96*96), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

def train_model(model, criterion, optimizer, scheduler, num_epoch = 10):
    since = time.time()
    
    best_mode_wts = copy.deepcopy(model.state_dict())
    best_loss = -1.0
    for epoch in range(num_epoch):
        print("Epoch {}/{}".format(epoch, num_epoch-1))
        print('-'*10)
        for phase in ['train']:
            if phase == 'train':
                scheduler.step()
                model.train()
            else:
                model.eval()
            
            run_loss = 0.0
            #run_correct = 0
            proc = 0
            
            for inputs, _ in dataloaders[phase]:
                proc += b_size
                
                inputs = inputs.to(device)
                #labels = labels.to(device)
                
                #zeros para
                optimizer.zero_grad()
                
                #forward
                #track history if and only if in training phase
                with torch.set_grad_enabled(phase == 'train'):
                    recon_x, x, mu, logvar = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    #print(outputs.size())
                    #print(preds)
                    loss = criterion(recon_x, x, mu, logvar)
                    
                    # backwarda and optimize only in training
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                if proc % 640 == 0:
                    print('processed photos are {}'.format(proc))
                # statistics
                run_loss += loss.item()*inputs.size(0)
                #run_correct += torch.sum(labels.data == preds)
                
                
            epoch_loss = run_loss/dataset_sizes[phase]
            #epoch_acc = run_correct/dataset_sizes[phase]
            
            print('{} Loss: {:.4f}'.format(phase, epoch_loss))
            
            # deep copy the model
            if best_loss < 0:
                best_loss = epoch_loss
                best_model_wts = copy.deepycopy(mode.state_dict())
                
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepycopy(mode.state_dict())
                
        print()
        
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed//60, time_elapsed%60))
    #print('Best Acc: {:.4f}'.format(best_acc))
        
    model.load(best_model_wts)
        
    return model

if __name__ == "__main__":
    
    save_path = './vae_linear.pt'
    model = Linear_Model()
    
    criterion = loss_function

    optimizer_ft = optim.SGD(model_ft.parameters(), lr = 0.001, momentum = 0.9)

    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size = 7, gamma = 0.1)

    model_best = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epoch = 10)
    torch.save(model_best.state_dict(), save_path)

Writing train.py


In [69]:
a = torch.ones(64, 1, 96, 96)
b = torch.zeros(64*1*96*96)
input_size = [*a.size()]
print(a.size())
print(b.size())
print(input_size)
dim = np.prod(input_size)
print(dim)
print(64*1*96*96)
input_size[1] = 2*input_size[1]
new_data = torch.cat([a.view(-1,dim), b.view(-1, dim)]).view(*input_size)
new_data.size()

torch.Size([64, 1, 96, 96])
torch.Size([589824])
[64, 1, 96, 96]
589824
589824


torch.Size([64, 2, 96, 96])

In [51]:
a.view(-1, dim).size()
b.size()

torch.Size([589824])

In [20]:
for batch_idx, (data, _) in enumerate(train_loader):
    if batch_idx == 0:
        print(data.size())
        #print(data)
        break

torch.Size([4, 3, 96, 96])


In [4]:
sample = torch.randn(64, 20)

In [7]:
sample.size()

torch.Size([64, 20])

In [8]:
a = torch.FloatTensor([1,2,3])
b = torch.FloatTensor([4,5,6])

In [10]:
b

tensor([ 4.,  5.,  6.])

In [15]:
torch.cat([a, b])

tensor([ 1.,  2.,  3.,  4.,  5.,  6.])

In [17]:
torch.exp(0.5*a)

tensor([ 1.6487,  2.7183,  4.4817])

In [18]:
torch.randn_like(a)

tensor([ 0.8990,  0.8012, -0.0942])

In [19]:
a

tensor([ 1.,  2.,  3.])

In [21]:
help torch.randn_like()

SyntaxError: invalid syntax (<ipython-input-21-c7a522a0593b>, line 1)

In [7]:
c = torch.cat([a[:8], b[:8]])

In [20]:
size = a.size()

In [10]:
d = torch.ones(64*96*96)

In [21]:
e = d.view(*size)

In [26]:
a.size()
a[:, 0:].size()

torch.Size([64, 1, 96, 96])

In [28]:
f = torch.cat([a[1,:, :, :], e[1,:, :, :]])
f.size()

torch.Size([2, 96, 96])

In [39]:
import numpy as np

In [40]:
np.prod(a.size())

589824

In [7]:
recon_x = torch.ones(128, 27648)
x = torch.ones(128, 27648)
F.binary_cross_entropy(recon_x, x.view(-1, 3*96*96), reduction = 'sum')

TypeError: binary_cross_entropy() got an unexpected keyword argument 'reduction'

In [8]:
print(torch.__version__)

0.4.0


In [10]:
where(x < 1)

NameError: name 'where' is not defined

In [14]:
mu = torch.ones(128, 27648)
torch.Tensor(torch.randn(mu.size()), requires_grad=False)

TypeError: new() received an invalid combination of arguments - got (Tensor, requires_grad=bool), but expected one of:
 * (torch.device device)
 * (tuple of ints size, torch.device device)
      didn't match because some of the keywords were incorrect: requires_grad
 * (torch.Storage storage)
 * (Tensor other)
 * (object data, torch.device device)
      didn't match because some of the keywords were incorrect: requires_grad
